In [2]:
file_config = {"preamble_len": 4,
                "preamble_text": ["\documentclass{article}", 
                                  "\usepackage{polski}", 
                                  "\usepackage[utf8]{inputenc}", 
                                  "\begin{document}"]}
tasks_config = {"Dynamika": {"Równia pochyła": [{"treść": [r"AAA \\", "BBB"], "linie": (7,8)}]}}

In [49]:
a = [1,2,3,4]
a[2:2] = [a[3]]
a[3:3] = []
l.insert(newindex, l.pop(oldindex))

[1, 2, 4, 3, 4]

In [52]:
l = [x+1 for x in range(4)]
l.insert(1, l.pop(3))
l

[1, 4, 2, 3]

In [156]:
from copy import deepcopy
import os
import shutil
from pdflatex import PDFLaTeX
import json
from random import randrange

class Config(object):
    root_dir = "."
    task_dir = os.path.join(root_dir, "zadania")
    solv_dir = os.path.join(root_dir, "rozwiązania")
    file_config = {"preamble_len": 4,
                   "preamble_text": [r"\documentclass{article}", 
                                     r"\usepackage{polski}", 
                                     r"\usepackage[utf8]{inputenc}", 
                                     r"\begin{document}"],
                    "end_text": [r"\end{document}"]}
    tasks_config_file = os.path.join(root_dir, "tasks_config.json")
    
class TasksBaseManager(object):
    
    def __init__(self, from_file=False, from_dirs=False, from_dict=False, dct={}):
                
        assert from_file or from_dirs or from_dict
        assert (from_dict == True and dct != {}) or from_dict == False
        if from_file:
            with open(Config.tasks_config_file, "rb") as f:
                a = f.read()
                self.tasks_config = json.loads(a)
        else:
            if from_dirs:
                self.tasks_config = {}
                self.update_config_from_dirs()
            else:
                self.tasks_config = dct
        
    def create_pdf_all(self, fn="zadania_wszystkie", odir="."):
        tex_file = deepcopy(Config.file_config["preamble_text"])
        for section, section_tasks in self.tasks_config.items():
            tex_file.append(r"\section{{ {0} }}".format(section))
            for subsection, subsection_tasks in section_tasks.items():
                tex_file.append(r"\subsection{{ {0} }}".format(subsection))
                for i, task in enumerate(subsection_tasks):
                    tex_file.append(r"\textbf{{Zadanie {0}}} \\".format(i+1))
                    tex_file += [x + r"\\" for x in task["treść"]]
        tex_file += Config.file_config["end_text"]
        fn = os.path.join(odir, fn)
        pdfl = pdflatex.PDFLaTeX.from_binarystring("\n".join(tex_file).encode('utf-8'), fn)
        pdf, log, cp = pdfl.create_pdf(keep_pdf_file=True)
    
    def create_pdf(self, tasks, fn="zadania", odir="."):
        tex_file = deepcopy(Config.file_config["preamble_text"])
        for sec, section_tasks in tasks.items():
            tex_file.append(r"\section{{ {0} }}".format(sec))
            for subsec, subsection_tasks in section_tasks.items():
                tex_file.append(r"\subsection{{ {0} }}".format(subsec))
                for task_num in subsection_tasks:
                    tex_file.append(r"\textbf{{Zadanie {0}}} \\".format(task_num))
                    tex_file += [x + r"\\" for x in self.tasks_config[sec][subsec][task_num-1]["treść"]]
        tex_file += Config.file_config["end_text"]
        out_fn = os.path.join(odir, "{0}_{1}".format(fn, randrange(10000)))
        pdfl = pdflatex.PDFLaTeX.from_binarystring("\n".join(tex_file).encode('utf-8'), out_fn)
        pdf, log, cp = pdfl.create_pdf(keep_pdf_file=True)                   
    
    def add_task(self, sec, subsec, fn, sol_fn, num=None):
        num -= 1
        with open(fn, "r") as f:
            content = f.readlines()
        name = fn.split("/")[-1]
        sol_name = sol_fn.split("/")[-1]
        sec_dct = self.tasks_config.get(sec)
        if sec_dct is None:
            self._add_section(sec)
        subsec_dct = self.tasks_config[sec].get(subsec)
        if subsec_dct is None:
            self._add_subsection(subsec)
        if num is None:
            self.tasks_config[sec][subsec].append({"treść": content, "nazwa": name})
            n = len(self.tasks_config[sec][subsec])
        else:
            self.tasks_config[sec][subsec][num:num] = [{"treść": content, "nazwa": name}]
            self._rename_files(Config.task_dir, sec, subsec, num)
            self._rename_files(Config.solv_dir, sec, subsec, num)
            n = num+1
        task_fn = os.path.join(Config.task_dir, sec, subsec, "{0}_{1}".format(n, name))
        shutil.move(fn, task_fn)
        sol_new_fn = os.path.join(Config.solv_dir, sec, subsec, "{0}_{1}".format(n, sol_name))
        shutil.move(sol_fn, sol_new_fn)
            
    def update_config_from_dirs(self):
        for sec in os.listdir(Config.task_dir):
            self.tasks_config[sec] = {}
            for subsec in os.listdir(os.path.join(Config.task_dir, sec)):
                self.tasks_config[sec][subsec] = []
                tasks = os.listdir(os.path.join(Config.task_dir, sec, subsec))
                sorted_tasks = sorted(tasks, key=self._sort_files_key)
                for i, task in enumerate(sorted_tasks):
                    task_fn = os.path.join(Config.task_dir, sec, subsec, task)
                    with open(task_fn, "r") as f:
                        content = f.readlines()
                    try:
                        num = int(task.split("_")[0])
                        name = task.split("_")[1]
                    except Exception as e:
                        name = task.split("_")[0]
                    self.tasks_config[sec][subsec].append({"treść": content, "nazwa": name})
                    new_fn = os.path.join(Config.task_dir, sec, subsec, "{0}_{1}".format(i+1, name))
                    os.rename(task_fn, new_fn)
        for sec in os.listdir(Config.solv_dir):
            for subsec in os.listdir(os.path.join(Config.solv_dir, sec)):
                solvs = os.listdir(os.path.join(Config.solv_dir, sec, subsec))
                sorted_solvs = sorted(solvs, key=self._sort_files_key)
                for i, solv in enumerate(sorted_solvs):
                    solv_fn = os.path.join(Config.solv_dir, sec, subsec, solv)
                    try:
                        num = int(solv.split("_")[0])
                        name = solv.split("_")[1]
                    except Exception as e:
                        name = solv.split("_")[0]
                    new_fn = os.path.join(Config.solv_dir, sec, subsec, "{0}_{1}".format(i+1, name))
                    os.rename(solv_fn, new_fn)
        self._update_cfg()
                    
    def rename_numbers(self, sec, subsec, old_num, new_num):
        self.tasks_config[sec][subsec].insert(new_num-1, self.tasks_config[sec][subsec].pop(old_num-1))
        if new_num > old_num:
            step=-1
            num = old_num + 1
            max_num = new_num + 1
        else:
            step = 1
            num = new_num
            max_num = old_num
        self._rename_files(Config.task_dir, sec, subsec, num, step=step, max_num=max_num)
        self._rename_files(Config.solv_dir, sec, subsec, num, step=step, max_num=max_num)
        name = self.tasks_config[sec][subsec][new_num-1]["nazwa"]
        old_n = "{0}_{1}".format(old_num, name)
        new_n = "{0}_{1}".format(new_num, name)
        old_fn = os.path.join(Config.task_dir, sec, subsec, old_n)
        new_fn = os.path.join(Config.task_dir, sec, subsec, new_n)
        print("old: " + old_fn)
        print("new: " + new_fn)
        os.rename(old_fn, new_fn)
    
    def _update_cfg(self):
        with open(Config.tasks_config_file, "w", encoding='utf8') as f:
            json.dump(self.tasks_config, f)
            
    def _rename_files(self, tdir, sec, subsec, num, step=1, max_num=None):
        for task in os.listdir(os.path.join(tdir, sec, subsec)):
            tnr = int(task.split("_")[0])
            if tnr >= num:
                if max_num is None or tnr < max_num:
                    new_t = "{0}_{1}".format(tnr+step, task.split("_")[1])
                    old_fn = os.path.join(tdir, sec, subsec, task)
                    new_fn = os.path.join(tdir, sec, subsec, new_t)
                    print("old: " + old_fn)
                    print("new: " + new_fn)
                    os.rename(old_fn, new_fn)
        
    def _sort_files_key(self, x):
        name = x.split("_")
        try:
            num = int(x[0])
        except Exception as e:
            num = x[0]
        return num
    
    def _add_section(self, sec):
        os.makedir(os.path.join(Config.tasks_dir, sec))
        self.tasks_config[sec] = {}
        
    def _add_subsection(self, sec, subsec):
        os.makedir(os.path.join(Config.tasks_dir, sec, subsec))
        self.tasks_config[sec][subsec] = []

# Tests

### write_pdf

In [106]:
tm = TasksBaseManager(from_file=True)
tm.create_pdf_all()

### write tasks

In [88]:
tm = TasksBaseManager(from_file=True)
tm.create_pdf({"Dynamika": {"Równia": [1,2], "Bryła": [1]}, "termoDynamika": {"Bryła termo": [1,2]}})

### tasks from dirs

In [157]:
tm = TasksBaseManager(from_dirs=True)
tm.create_pdf_all()
tm.tasks_config

{'termodynamika': {},
 'dynamika': {'lala': [{'treść': ['fsafsfsafdgdgas\n'], 'nazwa': 'pierr.tex'}],
  'bryla': [{'treść': ['fdfasd\n', 'gdsgds\n', 'saaa\n'], 'nazwa': 'dwa.tex'},
   {'treść': ['fass\n', 'dafassa\n', 'fasfs\n'], 'nazwa': 'pier.tex'},
   {'treść': [], 'nazwa': 'noweee.tex'},
   {'treść': [], 'nazwa': 'nowe.tex'}]}}

### add task

In [130]:
tm = TasksBaseManager(from_dirs=True)
tm.add_task("dynamika", "bryla", "noweee.tex", "sol_noweee.tex", 1)

### rename tasks

In [165]:
tm = TasksBaseManager(from_dirs=True)
tm.rename_numbers("dynamika", "bryla", 2, 4)

old: ./zadania/dynamika/bryla/3_nowe.tex
new: ./zadania/dynamika/bryla/2_nowe.tex
old: ./zadania/dynamika/bryla/4_dwa.tex
new: ./zadania/dynamika/bryla/3_dwa.tex
old: ./rozwiązania/dynamika/bryla/3_pier.tex
new: ./rozwiązania/dynamika/bryla/2_pier.tex
old: ./rozwiązania/dynamika/bryla/4_sol
new: ./rozwiązania/dynamika/bryla/3_sol
old: ./zadania/dynamika/bryla/2_pier.tex
new: ./zadania/dynamika/bryla/4_pier.tex
